In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#%%capture
!pip install bitsandbytes
!pip install transformers==4.34.0
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 33.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency res

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

2024-04-12 08:46:04.458381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 08:46:04.458474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 08:46:04.580322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch

In [6]:
pipe = pipeline(
    "text-generation",
    model="Jyotiyadav/copywriting",
    #tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

adapter_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Result1 for MenuDataset

In [9]:

prompt = f'''As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
generate the output 
### Input data:
Crema al mascarpone 
### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 88.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 39.06 MiB is free. Process 2626 has 14.71 GiB memory in use. Of the allocated memory 13.74 GiB is allocated by PyTorch, and 859.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Result2 for MenuDataset

In [ ]:
prompt = f'''
    As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
    generate the output 
    
    ### Input data:
    Biscotti di Pan di zenzero (gingerbread) 
    
    ### Output:
    '''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result3 for MenuDataset

In [ ]:

prompt = f'''
    As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
    generate the output 
    
    ### Input data:
    Tortino di cioccolato con cuore fondente 
    
    ### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result4 for SocialMedia

In [ ]:

prompt = f'''
As the social media manager for the restaurant, your task is to craft a compelling weekly post for the restaurant's Facebook and Instagram pages. Here are some guidelines to follow:

Your post should align with the specified input.
Incorporate 3-5 emojis, ensuring they are spaced out throughout the post. Avoid using more than one emoji every two sentences.
Limit the mention of menu items to a maximum of 1 or 2 products.
Maintain a tone that highlights the restaurant's strengths rather than focusing on direct marketing.
Include a Call to Action (CTA) that may include opening hours, address, telephone number, or WhatsApp number if available.
Mention the restaurant's online menu, accessible at www.restaurants.menu.
Optionally, include relevant hashtags at the end of the post using the '#' symbol.

### Input data:
Menu 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result5 for SocialMedia

In [ ]:

prompt = f'''
As the social media manager for the restaurant, your task is to craft a compelling weekly post for the restaurant's Facebook and Instagram pages. Here are some guidelines to follow:

Your post should align with the specified input.
Incorporate 3-5 emojis, ensuring they are spaced out throughout the post. Avoid using more than one emoji every two sentences.
Limit the mention of menu items to a maximum of 1 or 2 products.
Maintain a tone that highlights the restaurant's strengths rather than focusing on direct marketing.
Include a Call to Action (CTA) that may include opening hours, address, telephone number, or WhatsApp number if available.
Mention the restaurant's online menu, accessible at www.restaurants.menu.
Optionally, include relevant hashtags at the end of the post using the '#' symbol.

### Input data:
Menu 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result6 for SocialMedia

In [ ]:

prompt = f'''
[INST] As the social media manager for the restaurant, your task is to craft a compelling weekly post for the restaurant's Facebook and Instagram pages. Here are some guidelines to follow:

Your post should align with the specified input.
Incorporate 3-5 emojis, ensuring they are spaced out throughout the post. Avoid using more than one emoji every two sentences.
Limit the mention of menu items to a maximum of 1 or 2 products.
Maintain a tone that highlights the restaurant's strengths rather than focusing on direct marketing.
Include a Call to Action (CTA) that may include opening hours, address, telephone number, or WhatsApp number if available.
Mention the restaurant's online menu, accessible at www.restaurants.menu.
Optionally, include relevant hashtags at the end of the post using the '#' symbol.

### Input data:
Theme days 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result7 for Advertising

In [ ]:

prompt = f'''
As an advertising manager, your task is to generate campaign descriptions based on the provided Goal [Ideal customer,Offsite Age,The Newlyweds,The Anniversaries,The Vegan, The Vegetarian,The Single,The Graduating Student,The Saver,The Couples,Commuters,The Tourist] and buyer's persona's . 
Craft four descriptions tailored specifically for restaurants based on the provided goal and buyers persona. Incorporate hashtags, emojis, and persuasive call-to-action to drive engagement and conversions..
Your role is to provide three descriptions based on the campaign Name & Interest in a dictionary. 
 
 ### Input data:
 {'category': 'I Single', 'Personas': {'Age': '18-65', 'DemographicData': 'Divorced, Single, Separated, Widowed'}} 
 
### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result8 for Advertising

In [ ]:

prompt = f'''
As an advertising manager, your task is to generate campaign descriptions based on the provided Goal [Ideal customer,Offsite Age,The Newlyweds,The Anniversaries,The Vegan, The Vegetarian,The Single,The Graduating Student,The Saver,The Couples,Commuters,The Tourist] and buyer's persona's . 
Craft four descriptions tailored specifically for restaurants based on the provided goal and buyers persona. Incorporate hashtags, emojis, and persuasive call-to-action to drive engagement and conversions..
Your role is to provide three descriptions based on the campaign Name & Interest in a dictionary. 

### Input data:
{'category': 'The couples', 'Personas': {'Age': '18-65', 'Interests': 'Partner, Officially engaged, in a relationship, open relationship, married, civil union'}} 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])

### Result 9

In [ ]:

prompt = f'''
As an advertising manager, your task is to generate campaign descriptions based on the provided Goal [Ideal customer,Offsite Age,The Newlyweds,The Anniversaries,The Vegan, The Vegetarian,The Single,The Graduating Student,The Saver,The Couples,Commuters,The Tourist] and buyer's persona's . 
Craft four descriptions tailored specifically for restaurants based on the provided goal and buyers persona. Incorporate hashtags, emojis, and persuasive call-to-action to drive engagement and conversions..
Your role is to provide three descriptions based on the campaign Name & Interest in a dictionary. 

### Input data:
{'category': 'Commuters', 'Personas': {'Age': '18-65', 'Behaviors': 'Commuters'}} 

### Output:
'''
    
result = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=32000,
    temperature=0.1,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(result[0]['generated_text'])